In [1]:
from mne_bids import make_bids_basename
from pathlib import Path, PurePath
import mne
import os
import matplotlib.pyplot as plt
import pandas as pd
import matlab.engine
import numpy as np
import scipy as sp
import scipy.io
pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
from mne.time_frequency import psd_multitaper

In [157]:
%matplotlib 

Using matplotlib backend: Qt5Agg


In [45]:
subpath = Path('~','projects','CIFAR','data_bids', 'sub-08' ).expanduser()

In [162]:
subpath = Path('~','projects','CIFAR','data_bids', 'sub-00' ).expanduser()
elecinfo = subpath.joinpath('anat','electrodes_info.csv')
dfelec = pd.read_csv(elecinfo)
ROIs = dfelec['Brodman'].unique()
nROIs = len(ROIs)

In [172]:
# import re
# string = dfelec['electrode_name']
# m = re.search(r"\d", string)
# m.start()
# string[0:m.start()]

'Grid'

In [49]:
for i in range(0,9):
    subpath = Path('~','projects','CIFAR','data_bids', f'sub-0{i}' ).expanduser()
    elecinfo = subpath.joinpath('anat','electrodes_info.csv')
    dfelec = pd.read_csv(elecinfo)
    ROIs = dfelec['Brodman'].unique()
    nROIs = len(ROIs)
    print(ROIs)

['BA2' 'MT' 'V2' 'BA1' 'V1' 'BA3b' 'BA6' 'BA44' 'BA45' 'BA4a' 'perirhinal']
['BA45' 'perirhinal' 'BA44' 'MT' 'V2' 'V1' 'BA3a']
['BA1' 'BA2' 'BA3b' 'perirhinal' 'BA6' 'BA44' 'BA45' 'V2' 'MT' 'BA3a']
['MT' 'V2' 'BA2' 'BA6' 'perirhinal' 'BA45' 'BA44' 'BA3b']
['BA2' 'BA3b' 'BA6' 'MT' 'BA1' 'BA4a' 'BA3a' 'BA45' 'BA44' 'perirhinal'
 'V2' 'V1']
['BA3a' 'BA44' 'perirhinal' 'MT' 'BA45' 'BA6' 'V2' 'BA2' 'BA3b' 'BA1']
['MT' 'V2' 'V1' 'BA2' 'BA1' 'BA44' 'BA45' 'BA3b' 'BA6' 'BA4a' 'BA4p'
 'BA3a' 'perirhinal']
['MT' 'perirhinal' 'BA44' 'BA6' 'BA45' 'BA3a' 'BA2' 'V2' 'BA4a' 'BA3b'
 'BA1']
['MT' 'V2' 'perirhinal' 'V1' 'BA44' 'BA45' 'BA3b' 'BA6' 'BA2' 'BA1']


In [66]:
len(dfelec['isdepth'].loc[dfelec['Brodman']=='V2'].loc[dfelec['isdepth']==0])

8

In [53]:
ROI_info = {'Brodman': [], 'ndepth': [], 'nGrid/strip': []}
ROI_info['Brodman']

[]

In [122]:
for ROI in ROIs: 
    ROI_info['Brodman'].append(ROI)
    ROI_info['ndepth'].append(len(dfelec['isdepth'].loc[dfelec['Brodman']==ROI].loc[dfelec['isdepth']==1]))
    ROI_info['nsurface'].append(len(dfelec['isdepth'].loc[dfelec['Brodman']==ROI].loc[dfelec['isdepth']==0]))
    ROI_info['n_lh'].append(len(dfelec['hemisphere'].loc[dfelec['Brodman']==ROI].loc[dfelec['hemisphere']=='lh']))
    ROI_info['n_rh'].append(len(dfelec['hemisphere'].loc[dfelec['Brodman']==ROI].loc[dfelec['hemisphere']=='rh']))

In [138]:
#Per subjects
column_names = ['Brodman', 'ndepth', 'nsurface', 'n_lh', 'n_rh']
dfsub = pd.DataFrame(columns = column_names)
for i in range(0,9):
    subpath = Path('~','projects','CIFAR','data_bids', f'sub-0{i}' ).expanduser()
    elecinfo = subpath.joinpath('anat','electrodes_info.csv')
    dfelec = pd.read_csv(elecinfo)
    ROIs = dfelec['Brodman'].unique()
    nROIs = len(ROIs)
    ROI_info = {'Brodman': [], 'ndepth': [], 'nsurface': [], 'n_lh': [], 'n_rh': []}
    for ROI in ROIs: 
        ROI_info['Brodman'].append(ROI)
        ROI_info['ndepth'].append(len(dfelec['isdepth'].loc[dfelec['Brodman']==ROI].loc[dfelec['isdepth']==1]))
        ROI_info['nsurface'].append(len(dfelec['isdepth'].loc[dfelec['Brodman']==ROI].loc[dfelec['isdepth']==0]))
        ROI_info['n_lh'].append(len(dfelec['hemisphere'].loc[dfelec['Brodman']==ROI].loc[dfelec['hemisphere']=='lh']))
        ROI_info['n_rh'].append(len(dfelec['hemisphere'].loc[dfelec['Brodman']==ROI].loc[dfelec['hemisphere']=='rh']))
    dfROI_info = pd.DataFrame(data=ROI_info)
    dfsub = pd.concat([dfsub, dfROI_info])

,Brodman,ndepth,nsurface,n_lh,n_rh
0,BA2,0,17,0,17
1,MT,1,24,0,25
2,V2,10,37,5,42
3,BA1,0,8,0,8
4,V1,0,4,0,4
5,BA3b,0,1,0,1
6,BA6,0,2,1,1
7,BA44,0,3,0,3
8,BA45,0,2,0,2
9,BA4a,0,2,2,0


In [147]:
# Total 
dfROIs = {'Brodman': [], 'ndepth': [], 'nsurface': [], 'n_lh': [], 'n_rh': []}
ROIs = dfsub['Brodman'].unique()
for ROI in ROIs:
    dfROIs['Brodman'].append(ROI)
    dfROIs['ndepth'].append(sum(dfsub['ndepth'].loc[dfsub['Brodman']==ROI]))
    dfROIs['nsurface'].append(sum(dfsub['nsurface'].loc[dfsub['Brodman']==ROI]))
    dfROIs['n_lh'].append(sum(dfsub['n_lh'].loc[dfsub['Brodman']==ROI]))
    dfROIs['n_rh'].append(sum(dfsub['n_rh'].loc[dfsub['Brodman']==ROI]))

dfROIs = pd.DataFrame(data=dfROIs)

In [166]:
dfROIs.plot(x='Brodman', y= ['nsurface','ndepth','n_lh','n_rh'], kind='bar', title='Number of ECoG contacts per Brodman area summed over all subjects')

In [116]:
for ROI in ROIs: 
    ROI_info = {'Brodman': [], 'ndepth': [], 'nsurface': [], 'n_lh': [], 'n_rh': []}
    ROI_info['Brodman'].append(ROI)
    ROI_info['ndepth'].append(len(dfelec['isdepth'].loc[dfelec['Brodman']==ROI].loc[dfelec['isdepth']==1]))
    ROI_info['nsurface'].append(len(dfelec['isdepth'].loc[dfelec['Brodman']==ROI].loc[dfelec['isdepth']==0]))
    ROI_info['n_lh'].append(len(dfelec['hemisphere'].loc[dfelec['Brodman']==ROI].loc[dfelec['hemisphere']=='lh']))
    ROI_info['n_rh'].append(len(dfelec['hemisphere'].loc[dfelec['Brodman']==ROI].loc[dfelec['hemisphere']=='rh']))

In [141]:
len(dfsub['Brodman'].unique())

13

In [137]:
dfsub['Brodman'].unique()

array(['BA2', 'MT', 'V2', 'BA1', 'V1', 'BA3b', 'BA6', 'BA44', 'BA45',
       'BA4a', 'perirhinal', 11, 'BA3a', 7, 10, 8, 12, 'BA4p', 13],
      dtype=object)

In [54]:
dfelec['Brodman'].loc[dfelec['electrode_name'] == 'RTo7']

Series([], Name: Brodman, dtype: object)

In [7]:
dfelec['electrode_name'].loc[dfelec['Brodman'] == 'BA45']

26    RIa14
27    RIa13
28    RIa12
29    RIa11
30    RIa10
31     RIa9
32     RIa8
Name: electrode_name, dtype: object

In [ ]:
# Channels to pick
['RTo9-RTo10',  'RTo7-RTo8', 'RO1-RO2', 'RIp9-RIp10', 'RFp5-RFp6', 'RIa5-RIa6', 'RIa11-RIa12', 'RFPs9-RFPs10', 'RTp5-RTp6']

In [ ]:
proc = 'BP'
subid = 'JuRo'
cfsubdir = Path('~','CIFAR_data','iEEG_10','subjects', subid, 'EEGLAB_datasets', 'bipolar_montage').expanduser()
fname = helper.CIFAR_filename(subid=subid,task='rest_baseline',proc=proc)
fpath = cfsubdir.joinpath(fname)
fpath = os.fspath(fpath)
raw = mne.io.read_raw_eeglab(fpath)

In [ ]:
# ROIs table
